In [2]:
import pandas as pd

In [50]:
education_df = pd.read_excel("education.xlsx")

In [48]:
crimes_2020 = pd.read_csv("crimes2020.csv")
crimes_2021 = pd.read_csv("crimes2021.csv")
crimes_2022 = pd.read_csv("crimes2022.csv")
crimes_2023 = pd.read_csv("crimes2023.csv")
crimes_2024 = pd.read_csv("crimes2024.csv")

In [51]:
# Define the column translation mapping
column_mapping = {
    "שם רשות": "Settlement",
    "אשכול סוציו-אקונומי": "SocioeconomicGroup",
    "שם מחוז גיאוגרפי": "DistrictName",
    "מספר תושבים ברשות לשנת תשפג": "NumResidents",
    "מספר תלמידים גרים ברשות": "NumStudents",
    "שיעור התלמידים בחינוך טכנולוגי": "RateInTechEdu",
    "אחוז נשירה מלומדים": "DropoutRate",
    "אחוז זכאים לבגרות  מהלומדים": "EligibleForBagrutRate",
    "אחוז זכאות לבגרות  מצטיינת מהלומדים": "EligibleForExcellentBagrutRate",
    "אחוז זכאות 5 יחידות מתמטיקה": "5UnitsMathematicsRate"
}

# Keep only the required columns and rename them
education_df = education_df[list(column_mapping.keys())].rename(columns=column_mapping)

# Convert the relevant columns to percentages (multiply by 100 and format)
percentage_columns = ['RateInTechEdu', 'DropoutRate', 'EligibleForBagrutRate',
                      'EligibleForExcellentBagrutRate', '5UnitsMathematicsRate']

for col in percentage_columns:
    # Ensure the column is numeric
    education_df[col] = pd.to_numeric(education_df[col], errors='coerce')
    # Multiply by 100 to convert to percentage, round to 2 decimal places
    education_df[col] = (education_df[col] * 100).round(1)
    # Convert to string and add '%' symbol
    education_df[col] = education_df[col].apply(lambda x: f"{x:.1f}%" if pd.notnull(x) else "-")


# Display the first few rows
education_df.head()

,Settlement,SocioeconomicGroup,DistrictName,NumResidents,NumStudents,RateInTechEdu,DropoutRate,EligibleForBagrutRate,EligibleForExcellentBagrutRate,5UnitsMathematicsRate
0,אבו גוש,3,ירושלים,7942,1717,43.1%,0.7%,61.5%,8.5%,9.4%
1,אבו סנאן,3,צפון,14687,2573,48.2%,1.8%,83.9%,20.8%,14.3%
2,אבן יהודה,9,מרכז,14365,3508,16.2%,1.3%,92.6%,21.1%,30.3%
3,אום אל-פחם,2,חיפה,58665,13584,47.7%,0.4%,74.4%,18.2%,11.7%
4,אופקים,3,דרום,35506,7994,52.8%,0.8%,44.8%,2.1%,6.2%


In [52]:
# Count rows with null values
null_rows_count = education_df.isnull().any(axis=1).sum()
print(f"Number of rows with null values: {null_rows_count}")

# Count total number of rows in the DataFrame
total_rows = education_df.shape[0]
print(f"Total number of rows in the DataFrame: {total_rows}")

Number of rows with null values: 0
Total number of rows in the DataFrame: 255


In [53]:
# Column mapping and remove rows with null values
# Define column mapping for crime datasets
crime_column_mapping = {
    "Year": "Year",
    "Yeshuv": "Settlement",
    "StatisticGroupKod": "StatisticGroupKod",
    "StatisticGroup": "StatisticGroup"
}

# List of crime datasets
crime_dfs = ["crimes_2020", "crimes_2021", "crimes_2022", "crimes_2023", "crimes_2024"]

# Process each crime dataset
for crime_df_name in crime_dfs:
    crime_df = globals()[crime_df_name]  # Get the DataFrame by name
    crime_df = crime_df[list(crime_column_mapping.keys())].rename(columns=crime_column_mapping)
    crime_df = crime_df.dropna()  # Remove rows with null values
    globals()[crime_df_name] = crime_df  # Save the modified DataFrame back

# Display first few rows of one crime dataset as a check
crimes_2020.head()

,Year,Settlement,StatisticGroupKod,StatisticGroup
0,2020,נתניה,200,עבירות סדר ציבורי
1,2020,ראשון לציון,600,עבירות כלפי המוסר
2,2020,הרצליה,200,עבירות סדר ציבורי
4,2020,ירושלים,700,עבירות כלפי הרכוש
6,2020,ירושלים,400,עבירות נגד גוף


In [54]:
# Update specific Settlement names in education_df
education_df["Settlement"] = education_df["Settlement"].replace({
    "תל אביב-יפו": "תל אביב יפו",
    "פתח תקוה": "פתח תקווה"
})

In [55]:
from google.colab import files

# Function to group by 'Yeshuv' and 'StatisticGroup', counting occurrences
def preprocess_dataset(df):
    grouped = df.groupby(['Settlement', 'StatisticGroup', 'StatisticGroupKod', 'Year']).size().reset_index(name='Count')
    return grouped

# Apply the function to each dataset
grouped_2020 = preprocess_dataset(crimes_2020)
grouped_2021 = preprocess_dataset(crimes_2021)
grouped_2022 = preprocess_dataset(crimes_2022)
grouped_2023 = preprocess_dataset(crimes_2023)
grouped_2024 = preprocess_dataset(crimes_2024)

# Combine all grouped datasets into one
combined_grouped = pd.concat([grouped_2020, grouped_2021, grouped_2022, grouped_2023, grouped_2024], ignore_index=True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [56]:
# Merge 'combined_grouped' with 'education_df' on the common settlement names
merged_df = pd.merge(combined_grouped, education_df[['Settlement', 'NumResidents']],
                     left_on='Settlement', right_on='Settlement',
                     how='left')

# Save the merged dataset to a new CSV file
output_file_with_residents = 'merged_crimes_with_residents.csv'
merged_df.to_csv(output_file_with_residents, index=False, encoding='utf-8-sig')

# Drop rows where 'StatisticGroupKod' is -1 or 'NumResidents' is NaN
merged_df_cleaned = merged_df.dropna(subset=['NumResidents'])
merged_df_cleaned = merged_df_cleaned[merged_df_cleaned['StatisticGroupKod'] != -1]

# Add 'CrimeRate' column by dividing 'Count' by 'NumResidents' and converting to percentage
merged_df_cleaned['CrimeRate'] = (merged_df_cleaned['Count'] / merged_df_cleaned['NumResidents']) * 100
merged_df_cleaned['CrimeRate'] = (merged_df_cleaned['CrimeRate'].round(2).astype(str) + '%')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [57]:
# Calculate total crime counts for each year and settlement
total_crime_counts = merged_df_cleaned.groupby(['Year', 'Settlement'], as_index=False)['Count'].sum()

# Merge with NumResidents
total_crime_counts = pd.merge(total_crime_counts, education_df[['Settlement','NumResidents']],
                              left_on='Settlement', right_on='Settlement', how='left')

# Calculate the total crime rate (CrimeRate = Count / NumResidents)
total_crime_counts['CrimeRate'] = (total_crime_counts['Count'] / total_crime_counts['NumResidents']) * 100
total_crime_counts['CrimeRate'] = total_crime_counts['CrimeRate'].round(1).astype(str) + '%'

# Set StatisticGroup as 'Total Crime' and StatisticGroupKod as a new value (e.g., -2 to signify total crime)
total_crime_counts['StatisticGroup'] = 'כל העבירות'
total_crime_counts['StatisticGroupKod'] = -2  # Arbitrary code for total crime

# Add the total crime rows to the original dataset, ensuring they appear in the correct position
# Sort by Year and Settlement so the total crime rows are added correctly
total_crime_counts_sorted = total_crime_counts.sort_values(by=['Year', 'Settlement'])

# Merge total crime rows into the original dataset
merged_df_with_totals = pd.concat([merged_df_cleaned, total_crime_counts_sorted[['Year', 'Settlement', 'StatisticGroup', 'StatisticGroupKod', 'Count', 'NumResidents', 'CrimeRate']]], ignore_index=True)

# Sort the final dataset by Year and Settlement so the total crime rows are properly placed
merged_df_with_totals = merged_df_with_totals.sort_values(by=['Year', 'Settlement']).reset_index(drop=True)

# Merge merged_df_with_totals and education_df on the 'Settlement' column
merged_df_with_totals = merged_df_with_totals.merge(education_df[['Settlement', 'DistrictName']],
                                                     on='Settlement',
                                                     how='left')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [59]:
# Filter rows that do not contain '-' in the specified columns
education_df = education_df[~education_df[percentage_columns].isin(['-']).any(axis=1)]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>